In [1]:
import numpy as np
import feather
from ball_tree import BallTree  # TODO: force python to only look locally for import
from faster_sandwich_filling import multiply_XeeX, CutoffError, GeographyError, get_kernel_fn
from numpy.testing import assert_allclose
from patsy import dmatrices, dmatrix
from core import parse_lat_long, check_parameters
from scipy.sparse import csr_matrix, lil_matrix

In [2]:
def conley_cross_section(formula_like, data, lat_long, cutoff, kernel = 'uniform'):
    """Calculate Conley standard errors for a cross section.

    Parameters
    ----------
    formula_like : string or other Patsy formula
        e.g. "my_y_variable = my_X_var1 + my_X_var2"
        See http://patsy.readthedocs.io/en/latest/formulas.html#formulas for
        details on Patsy formulas.
    data : array-like
        Must contain all the variables referenced in the formula.
    lat_long : array_like, or tuple of names of columns in data
        An N-by-2 array of latitudes (in the first column) and longitudes (in
        the second column). Both latitude and longitude should be measured
        in degrees. Valid longitudes are [-90, 90].  Valid latitudes are
        (-180, 180]. The number of rows should be the same as the rows in data.
    cutoff : number
        The maximum distance over which covariance is possible.
        cutoff must be a positive number in the range (0, 20015).
    kernel : string
        The kernel function to weight the distances by. Valid options are:
        'bartlett', 'triangle', 'epanechnikov', 'quartic', 'biweight' and
        'triweight'. (Bartlett is the same as triangle. Quartic is the same as
        biweight.)
    """
    y, X = dmatrices(formula_like, data, eval_env = 1, NA_action = 'raise')
    # TODO: handle cases where people provide weird formulas?

    lat_long = parse_lat_long(lat_long, data)
    # Raise an exception if the data look funky
    nobs = check_parameters(y, X, lat_long, cutoff)

    # I have no idea if this leaf_size is reasonable.  If running out of memory,
    # divide N by a larger number.
    # 40 is the default.
    leaf_size = max(40, nobs // 1000)
    # TODO: consider a more sophisticated way of calculating residuals (e.g. one that
    # allows for fancy fixed effects)
    betahat, _, rank, _ = np.linalg.lstsq(X, y)
    if rank != X.shape[1]:
        raise np.linalg.LinAlgError('X matrix is not full rank!')
    del rank
    residuals = (y - X @ betahat)[0]
    balltree = BallTree(lat_long, metric = 'greatcircle', leaf_size = leaf_size)
    if kernel == 'uniform':
        neighbors = balltree.query_radius(lat_long, r = cutoff)
        filling = multiply_XeeX(neighbors, residuals, X, kernel)
    else:
        neighbors, neighbor_distances = balltree.query_radius(
            lat_long, r = cutoff, return_distance = True)
        filling = multiply_XeeX(neighbors, residuals, X, kernel,
                                distances = neighbor_distances, cutoff = cutoff)
        del neighbor_distances
    del balltree, neighbors, y, residuals

    bread = np.linalg.inv(X.T @ X)
    sandwich = nobs * (bread.T @ filling @ bread)
    se = np.sqrt(np.diag(sandwich)).reshape(-1, 1)
    return se



In [3]:
def test_quakes():
    quakes = feather.read_dataframe('tests/datasets/quakes.feather')
    quakes_lat = quakes['lat'].reshape(-1, 1)
    # Subtract 180 because they've done 0 to 360.  See:
    # https://stackoverflow.com/questions/19879746/why-are-datasetquakes-longtitude-values-above-180
    quakes_long = quakes['long'].reshape(-1, 1) - 180
    quakes_lat_long = np.hstack((quakes_lat, quakes_long))
    cutoff = 100

    # correct_results = conley_unfancy(quakes_y, quakes_X, quakes_lat_long, cutoff)
    correct_results = np.array((108.723235, 19.187791)).reshape(-1, 1)  # faster testing
    fast_results = conley_cross_section("depth ~ mag", quakes,
                                        quakes_lat_long, cutoff)
    assert_allclose(correct_results, fast_results)
# test_quakes()

IndexError: Out of bounds on buffer access (axis 0)

In [5]:
quakes = feather.read_dataframe('tests/datasets/quakes.feather')
quakes_lat = quakes['lat'].reshape(-1, 1)
quakes_long = quakes['long'].reshape(-1, 1) - 180
quakes_lat_long = np.hstack((quakes_lat, quakes_long))
cutoff = 100

balltree = BallTree(quakes_lat_long, metric = 'greatcircle')
neighbors, distances = balltree.query_radius(quakes_lat_long, r = cutoff, return_distance = True)

    
y, X = dmatrices(data=quakes, formula_like='depth ~ mag')
betahat, _, rank, _ = np.linalg.lstsq(X, y)
if rank != X.shape[1]:
    raise np.linalg.LinAlgError('X matrix is not full rank!')
del rank
residuals = (y - X @ betahat)


In [ ]:
def conley_unfancy(y, X, lat_long, cutoff):
    N = y.shape[0]
    k = X.shape[1]
    bread = np.linalg.inv(X.T @ X)  # 'bread' in the sandwich-estimator sense

    # Run OLS to get residuals
    betahat = bread @ X.T @ y  # '@' is matrix multiplication, equivalent to np.dot
    residuals = y - X @ betahat
    meat_matrix = np.zeros((k, k))
    row_of_ones = np.ones((1, N))
    column_of_ones = np.ones((k, 1))
    # every_point_is_a_neighbor_of_every_other = True
    for i in range(N):
        dist = great_circle_one_to_many(lat_long, lat_long[i])

        window = dist <= cutoff
        # if not all(window):
        #     every_point_is_a_neighbor_of_every_other = False
        X_i = X[i, ].reshape(-1, 1)
        residuals_i = residuals[i, ].reshape(-1, 1)

        #         k x 1       1 x n        1 x 1
        XeeXh = (((X_i @ row_of_ones * residuals_i) *
                  (column_of_ones @ (residuals.T * window.T))) @ X)
        #                 k x 1                1 x n            n x k
        meat_matrix += XeeXh
    meat_matrix = meat_matrix / N

    sandwich = N * (bread.T @ meat_matrix @ bread)
    se = np.sqrt(np.diag(sandwich)).reshape(-1, 1)
    return se


In [98]:

def neighbors_to_sparse_nonuniform(neighbors, distances, kernel, cutoff):
    nrow = neighbors.shape[0]
    if distances is None:
        raise ValueError('You must provide distances if using a non-uniform kernel')
    if cutoff is None:
        raise ValueError('You must provide a cutoff if using a non-uniform kernel')
    if distances.shape[0] != nrow:
        raise ValueError('neighbors and distances have different numbers of rows')
        
    neighbors_lil = lil_matrix((nrow, nrow), dtype=np.float_)
    kernel_fn = get_kernel_fn(kernel)
    for i, neighbor_list in enumerate(neighbors):
        assert neighbor_list.shape == distances[i].shape
        # fun fact: you have to apply the kernel function here; you can't do a sparse matrix 
        # of distances, since some distances (e.g. distance with self) are zero.
        neighbors_lil[i, neighbor_list] = kernel_fn(distances[i], cutoff)
    return neighbors_lil.tocsr()


def neighbors_to_sparse_uniform(neighbors):
    nrow = neighbors.shape[0]
    neighbors_lil = lil_matrix((nrow, nrow), dtype=np.float_)
    for i, neighbor_list in enumerate(neighbors):
        neighbors_lil[i, neighbor_list] = 1
    return neighbors_lil.tocsr()


def neighbors_to_sparse(neighbors, kernel = 'uniform', distances = None, cutoff = None):
    if kernel == 'uniform':
        if cutoff is not None or distances is not None:
            raise ValueError("this combination of parameters should never be necessary; it's a coding mistake")
        return neighbors_to_sparse_uniform(neighbors)
    else:
        return neighbors_to_sparse_nonuniform(neighbors, distances, kernel, cutoff)

neighbors_sp = neighbors_to_sparse(neighbors)
neighbors_sp[1,100]


nrow = residuals.shape[0]
# not smart enough to convert an (N, 1) array to a (N,) array, so manually reshape
resid_diag_matrix = diags(residuals.reshape(-1), offsets=0, shape = (nrow, nrow))
resid_x_neighbors = neighbors_sp * resid_diag_matrix

# for i in range(residuals.shape[0]):
#     resid_x_neighbors[i, :] *= residuals



def test_mult_uniform(X, residuals, neighbors):
    N = X.shape[0]
    k = X.shape[1]
    
    
    meat_matrix = np.zeros((k, k))
    row_of_ones = np.ones((1, N))
    column_of_ones = np.ones((k, 1))
    neighbors_sp = neighbors_to_sparse(neighbors)
    
    neighbors_dense = neighbors_sp.toarray()
    for i in range(N):
        window = neighbors_dense[i, :]
        X_i = X[i, ].reshape(-1, 1)
        residuals_i = residuals[i, ].reshape(-1, 1)
        
        #         k x 1       1 x n        1 x 1
        XeeXh = (((X_i @ row_of_ones * residuals_i) *
                  (column_of_ones @ (residuals.T * window.T))) @ X)
        #                 k x 1                1 x n            n x k
        meat_matrix += XeeXh
    correct = meat_matrix / N
    
    # I want element-wise multiplication of the residuals vector by the neighbors weights matrix.
    # Sparse matrices don't have element-wise multiplication, but it's equivalent to cast 
    # the residuals as a sparse diagonal matrix, then do matrix multiplication.
    # The diags function isn't smart enough to convert an (N, 1) array to a (N,) array, so manually reshape.
    # Then, unlike numpy arrays, with sparse matrices, '*' means matrix multiplication, NOT element-wise.
    resid_diag_matrix = diags(residuals.reshape(-1), offsets = 0, shape = (nrow, nrow))
    resid_x_neighbors = neighbors_sp * resid_diag_matrix
    proposed = (X.T @ resid_x_neighbors @ X) / N
    
    bread = np.linalg.inv(X.T @ X)
    correct_sandwich = N * (bread.T @ correct @ bread)
    correct_se = np.sqrt(np.diag(correct_sandwich)).reshape(-1, 1)
    proposed_sandwich = N * (bread.T @ proposed @ bread)
    proposed_se = np.sqrt(np.diag(proposed_sandwich)).reshape(-1, 1)
    
    
    np.testing.assert_allclose(correct_se, proposed_se)
    
    
test_mult_uniform(X, residuals, neighbors)
# test_mult(X, residuals, neighbors, distances, kernel = 'epanechnikov', cutoff = 100)

#type(neighbors_sp)
from scipy.sparse import diags
#print(resid_x_neighbors, neighbors_sp)
# at_mult = 
# star_mult = X.T * resid_x_neighbors * X

# print(at_mult.shape)
# at_mult, star_mult)
# residuals.reshape(-1).shape

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

x and y nan location mismatch:
 x: DesignMatrix([[ 108.723235],
       [  19.187791]])
 y: DesignMatrix([[ 150.002453],
       [        nan]])

In [93]:
1000**0.5

31.622776601683793

In [8]:
#%load_ext Cython



def test_neighbors_to_sparse(neighbors, distances, cutoff):
    from scipy.sparse import find
    kernel = 'epanechnikov'  # just picked one for testing
    neighbors_sparse_nodistance = neighbors_to_sparse(neighbors)
    neighbors_sparse_withdistance = neighbors_to_sparse(neighbors, kernel, distances, cutoff)
    kernel_fn = get_kernel_fn(kernel)
    for i in range(neighbors.shape[0]):
        neighbors_row_argsort = np.argsort(neighbors[i])  # get the indexes that will sort the row
        neighbors_row_sorted = neighbors[i][neighbors_row_argsort]
        
        distance_row_sorted = kernel_fn(distances[i][neighbors_row_argsort], cutoff)
        
        # test that the neighbor indexes are the same
        np.testing.assert_equal(find(neighbors_sparse_nodistance.getrow(i))[1], neighbors_row_sorted)
        np.testing.assert_equal(find(neighbors_sparse_withdistance.getrow(i))[1], neighbors_row_sorted)
        # test that the distance weight values are the same
        np.testing.assert_equal(find(neighbors_sparse_withdistance.getrow(i))[2], distance_row_sorted)
test_neighbors_to_sparse(neighbors, distances, 100)
